# 🗂️ Meeting Notes Optimization Pipeline

This notebook builds a complete data engineering pipeline to process internal meeting notes:
- Clean and structure raw meeting text
- Engineer LLM prompts
- Generate AI summaries (with Hugging Face)
- Reformat into natural language
- Upload final output to **BigQuery**


## 📥 Load & Inspect Data

In [1]:
import pandas as pd

In [ ]:
df = pd.read_csv('Structured_Meeting_Notes.csv')
df.head()

,date,speaker,note,project,action_item,deadline
0,2025-03-28,Jordan,[Project: AI assistant integration] Jordan agr...,AI assistant integration,Set up Airflow DAGs,Friday
1,2025-04-04,Jordan,[Project: Internal documentation cleanup] Jord...,Internal documentation cleanup,Test Gemini with internal queries,Friday
2,2025-03-24,Jordan,[Project: Data pipeline improvement] Jordan hi...,Data pipeline improvement,Organize guild workshop,Monday
3,2025-02-13,Omar,[Project: Knowledge sharing guilds] Omar highl...,Knowledge sharing guilds,Summarize feedback for product,Thursday
4,2025-03-27,Anaïs,[Project: Dashboard redesign] Anaïs requested ...,Dashboard redesign,Summarize feedback for product,Thursday


## 🧹 Clean & Preprocess Data

### Check for duplicates

In [3]:
df_clean = df.dropna(subset=['note'])


### Capital letter for strings in ```speaker``` and ```project```

In [4]:
df_clean['speaker'] = df_clean['speaker'].str.title()
df_clean['project'] = df_clean['project'].str.title()
df_clean.head()

,date,speaker,note,project,action_item,deadline
0,2025-03-28,Jordan,[Project: AI assistant integration] Jordan agr...,Ai Assistant Integration,Set up Airflow DAGs,Friday
1,2025-04-04,Jordan,[Project: Internal documentation cleanup] Jord...,Internal Documentation Cleanup,Test Gemini with internal queries,Friday
2,2025-03-24,Jordan,[Project: Data pipeline improvement] Jordan hi...,Data Pipeline Improvement,Organize guild workshop,Monday
3,2025-02-13,Omar,[Project: Knowledge sharing guilds] Omar highl...,Knowledge Sharing Guilds,Summarize feedback for product,Thursday
4,2025-03-27,Anaïs,[Project: Dashboard redesign] Anaïs requested ...,Dashboard Redesign,Summarize feedback for product,Thursday


## 📆 Parse Deadlines into Dates

### Map the day of the week to the actual deadline date in datetime

In [6]:
from datetime import datetime, timedelta

def convert_weekday_to_date(row):
    weekdays = {
        "Monday": 0, "Tuesday": 1, "Wednesday": 2,
        "Thursday": 3, "Friday": 4, "Saturday": 5, "Sunday": 6
    }
    try:
        note_date = datetime.strptime(row["date"], "%Y-%m-%d") + timedelta(days=1)  # Shift by 1 day
        target_weekday = weekdays.get(row["deadline"])
        if target_weekday is None:
            return None
        days_ahead = (target_weekday - note_date.weekday()) % 7
        return (note_date + timedelta(days=days_ahead)).strftime("%Y-%m-%d")
    except:
        return None

df_clean.head()


,date,speaker,note,project,action_item,deadline
0,2025-03-28,Jordan,[Project: AI assistant integration] Jordan agr...,Ai Assistant Integration,Set up Airflow DAGs,Friday
1,2025-04-04,Jordan,[Project: Internal documentation cleanup] Jord...,Internal Documentation Cleanup,Test Gemini with internal queries,Friday
2,2025-03-24,Jordan,[Project: Data pipeline improvement] Jordan hi...,Data Pipeline Improvement,Organize guild workshop,Monday
3,2025-02-13,Omar,[Project: Knowledge sharing guilds] Omar highl...,Knowledge Sharing Guilds,Summarize feedback for product,Thursday
4,2025-03-27,Anaïs,[Project: Dashboard redesign] Anaïs requested ...,Dashboard Redesign,Summarize feedback for product,Thursday


## 🤖 Create LLM Input from Note Data

In [7]:
df_clean["deadline_date"] = df_clean.apply(convert_weekday_to_date, axis=1)
df_clean["deadline_date"] = pd.to_datetime(df_clean["deadline_date"])
df_clean["date"] = pd.to_datetime(df_clean["date"])



In [8]:
df_clean["days_until_deadline"] = (df_clean["deadline_date"] - df_clean["date"]).dt.days
df_clean.head()

,date,speaker,note,project,action_item,deadline,deadline_date,days_until_deadline
0,2025-03-28,Jordan,[Project: AI assistant integration] Jordan agr...,Ai Assistant Integration,Set up Airflow DAGs,Friday,2025-04-04,7
1,2025-04-04,Jordan,[Project: Internal documentation cleanup] Jord...,Internal Documentation Cleanup,Test Gemini with internal queries,Friday,2025-04-11,7
2,2025-03-24,Jordan,[Project: Data pipeline improvement] Jordan hi...,Data Pipeline Improvement,Organize guild workshop,Monday,2025-03-31,7
3,2025-02-13,Omar,[Project: Knowledge sharing guilds] Omar highl...,Knowledge Sharing Guilds,Summarize feedback for product,Thursday,2025-02-20,7
4,2025-03-27,Anaïs,[Project: Dashboard redesign] Anaïs requested ...,Dashboard Redesign,Summarize feedback for product,Thursday,2025-04-03,7


## 🔍 Clean Prompt Text (Regex Removal of Project Tags)

In [12]:
import pandas as pd
from transformers import pipeline

# Create the LLM input column (using your cleaned notes, now without time in date)
df_clean["llm_input"] = df_clean.apply(
    lambda row: f"Date: {pd.to_datetime(row['date']).strftime('%B %d, %Y')}, about '{row['project']}' led by {row['speaker']}: {row['note']}",
    axis=1
)

# Initialize the summarizer (using a pre-trained transformer model for summarization)
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

# Apply the transformer summarizer to the 'llm_input' column to generate natural summaries
df_clean["summary"] = df_clean["llm_input"].apply(
    lambda x: summarizer(x, max_length=50, min_length=15, do_sample=False)[0]["summary_text"]
)

# Show the final DataFrame with summaries
df_clean.head(10)



/opt/anaconda3/envs/malt-pipeline/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use mps:0


,date,speaker,note,project,action_item,deadline,deadline_date,days_until_deadline,llm_input,summary
0,2025-03-28,Jordan,[Project: AI assistant integration] Jordan agr...,Ai Assistant Integration,Set up Airflow DAGs,Friday,2025-04-04,7,"Date: March 28, 2025, about 'Ai Assistant Inte...",Concerns were raised about the scalability of...
1,2025-04-04,Jordan,[Project: Internal documentation cleanup] Jord...,Internal Documentation Cleanup,Test Gemini with internal queries,Friday,2025-04-11,7,"Date: April 04, 2025, about 'Internal Document...","Date: April 04, 2025, about 'Internal Documen..."
2,2025-03-24,Jordan,[Project: Data pipeline improvement] Jordan hi...,Data Pipeline Improvement,Organize guild workshop,Monday,2025-03-31,7,"Date: March 24, 2025, about 'Data Pipeline Imp...",Jordan highlighted the need for the data pipe...
3,2025-02-13,Omar,[Project: Knowledge sharing guilds] Omar highl...,Knowledge Sharing Guilds,Summarize feedback for product,Thursday,2025-02-20,7,"Date: February 13, 2025, about 'Knowledge Shar...",The importance of cross-departmental collabor...
4,2025-03-27,Anaïs,[Project: Dashboard redesign] Anaïs requested ...,Dashboard Redesign,Summarize feedback for product,Thursday,2025-04-03,7,"Date: March 27, 2025, about 'Dashboard Redesig...","The group brainstormed possible solutions, pr..."
5,2025-03-30,Louis,[Project: Malty AI latency issue] Louis raised...,Malty Ai Latency Issue,Run dbt models in staging,Sunday,2025-04-06,7,"Date: March 30, 2025, about 'Malty Ai Latency ...",Concerns were raised about scalability of the...
6,2025-03-02,Mélanie,[Project: Malty AI latency issue] Mélanie agre...,Malty Ai Latency Issue,Review latency logs,Sunday,2025-03-09,7,"Date: March 02, 2025, about 'Malty Ai Latency ...",Concerns were raised about the scalability of...
7,2025-02-28,Jordan,[Project: Knowledge sharing guilds] Jordan rai...,Knowledge Sharing Guilds,Share documentation with the team,Friday,2025-03-07,7,"Date: February 28, 2025, about 'Knowledge Shar...",Jordan raised concerns about the knowledge sh...
8,2025-03-31,Louis,[Project: Freelancer matching algorithm] Louis...,Freelancer Matching Algorithm,Summarize feedback for product,Monday,2025-04-07,7,"Date: March 31, 2025, about 'Freelancer Matchi...","The group brainstormed possible solutions, pr..."
9,2025-02-24,Louis,[Project: Freelancer matching algorithm] Louis...,Freelancer Matching Algorithm,Review latency logs,Monday,2025-03-03,7,"Date: February 24, 2025, about 'Freelancer Mat...",Louis requested a report on the freelancer ma...


## ✅ Upload to BigQuery

In [15]:

from google.oauth2 import service_account
from pandas_gbq import to_gbq

# 🔐 Use your downloaded JSON key here
credentials = service_account.Credentials.from_service_account_file(
    "gcp-key-pipeline-notes.json"  # <-- replace with the exact filename if different
)

# 🚀 Upload to BigQuery
to_gbq(
    dataframe=df_clean,
    destination_table="meeting_notes.summarized_notes",  # dataset.table
    project_id="pipeline-notes-optimizer",
    credentials=credentials,
    if_exists="replace"  # or "append" if you're adding more data later
)


100%|██████████| 1/1 [00:00<00:00, 4655.17it/s]


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## Queries

In [ ]:
from google.cloud import bigquery
from google.oauth2 import service_account

client = bigquery.Client(credentials=credentials, project=credentials.project_id)

Deadlines between 2025-03-15 and 2025-03-30:

In [24]:
query = """
    SELECT project, speaker, action_item, deadline_date
    FROM `pipeline-notes-optimizer.meeting_notes.summarized_notes`
    WHERE deadline_date BETWEEN '2025-03-15' AND '2025-03-30'
    ORDER BY deadline_date ASC
"""

df_query = client.query(query).to_dataframe()
df_query.head()

/opt/anaconda3/envs/malt-pipeline/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1933: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,project,speaker,action_item,deadline_date
0,Data Pipeline Improvement,Jordan,Organize guild workshop,2025-03-15
1,Ai Assistant Integration,Omar,Redesign UI mockups,2025-03-17
2,Data Pipeline Improvement,Jordan,Review latency logs,2025-03-18
3,Data Pipeline Improvement,Théo,Summarize feedback for product,2025-03-18
4,Dashboard Redesign,Jordan,Set up Airflow DAGs,2025-03-18


How many times did the most frequent speakers intervened?

In [26]:
query = """
    SELECT speaker, COUNT(*) AS entry_count
    FROM `pipeline-notes-optimizer.meeting_notes.summarized_notes`
    GROUP BY speaker
    ORDER BY entry_count DESC
"""

df_query = client.query(query).to_dataframe()
df_query.head()

/opt/anaconda3/envs/malt-pipeline/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1933: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,speaker,entry_count
0,Jordan,11
1,Omar,7
2,Louis,7
3,Anaïs,6
4,Théo,5


Get the average day until deadline:

(Output is always 7 because generated synthethic data only has 7-days deadline)

In [31]:
query = """
    SELECT project, AVG(days_until_deadline) AS avg_days_until_deadline
    FROM `pipeline-notes-optimizer.meeting_notes.summarized_notes`
    GROUP BY project
    ORDER BY avg_days_until_deadline ASC
"""

df_query = client.query(query).to_dataframe()
df_query.head()

/opt/anaconda3/envs/malt-pipeline/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1933: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,project,avg_days_until_deadline
0,Ai Assistant Integration,7.0
1,Internal Documentation Cleanup,7.0
2,Knowledge Sharing Guilds,7.0
3,Freelancer Matching Algorithm,7.0
4,Llm Experimentation,7.0


Project with the most actions:

In [30]:
query = """
    SELECT project, COUNT(action_item) AS total_action_items
    FROM `pipeline-notes-optimizer.meeting_notes.summarized_notes`
    GROUP BY project
    ORDER BY total_action_items DESC
    """

df_query = client.query(query).to_dataframe()
df_query.head()

/opt/anaconda3/envs/malt-pipeline/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1933: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,project,total_action_items
0,Data Pipeline Improvement,8
1,Ai Assistant Integration,7
2,Knowledge Sharing Guilds,7
3,Freelancer Matching Algorithm,7
4,Dashboard Redesign,6
